In [1]:
import ee
import geemap
from pathlib import Path

In [2]:
ee.Authenticate()

Enter verification code:  4/1AQSTgQFe7qfEsk4fwOmE3n0ufkUDeKQCzDmDXkWiKG0zcf8Zeu1YEUJNSVc



Successfully saved authorization token.


In [3]:
ee.Initialize()

Création d'une ee.Geometry() contenant la région Aquitaine (1 seul polygone)

In [11]:
ROI_FILE = Path().resolve().parent / "geometries" / "aquitaine.geojson"
roi = geemap.geojson_to_ee(str(ROI_FILE))

In [12]:
map = geemap.Map()
# map.add_basemap("Esri.WorldImagery")
map.add_basemap("Stadia.AlidadeSmoothDark")
map.centerObject(roi)

In [13]:
DIST_ALERT_RESOURCE = "projects/glad/HLSDIST/current"
veg_dist_status = ee.ImageCollection(f"{DIST_ALERT_RESOURCE}/VEG-DIST-STATUS").mosaic()

In [14]:
veg_dist_confirmed_alerts_mask = veg_dist_status.where(veg_dist_status.eq(6).Or(veg_dist_status.eq(8)), 1)
veg_dist_confirmed_alerts = veg_dist_confirmed_alerts_mask.selfMask()

In [15]:
roi_fc = ee.FeatureCollection(roi)
empty_img = ee.Image().byte()
roi_contour = empty_img.paint(**{"featureCollection": roi_fc, "color": 1, "width": 3})
map.addLayer(roi_contour, {"palette": "blue"}, "Aquitaine")

In [16]:
map.addLayer(veg_dist_confirmed_alerts, {"min": 6, "max": 8, "palette": ["orange", "red"]}, "VEG-DIST-CONFIRMED-ALERTS")

In [17]:
map

Map(center=[44.97663510825801, 0.40994910693179953], controls=(WidgetControl(options=['position', 'transparent…